### Notebook to calculate image-by-image zeropoints. 

In [1]:
#standard imports
%matplotlib inline
import numpy as np

from astroquery.vizier import Vizier
import astropy.coordinates as coords
from astropy.coordinates import Angle
import astropy.units as u

import matplotlib.pyplot as plt
import seaborn as sea

Vizier.ROW_LIMIT = 999999999

def write_reg(ra,dec,size='2"',color='green',shape='circle',filename='regions.reg', width=1):
    f = open(filename,'w')
    f.write('fk5\n')
    for i,j in zip(ra,dec):
        f.write(shape+' '+str(i)+' '+str(j)+' '+size+' # color='+color+ ' width='+str(width)+'\n')
    f.close()     

### Vizier query to select all SDSS sources about the images. 

In [ ]:
#specify the ra and dec to search about. Just pick coordinates in the middle of the image.
search_ra = 202.09615 *u.degree
search_dec = 21.000972 * u.degree

#specify the search radius, just pick big enough region. Needs to be bigger for CFHT imaging compared to SDSS.
search_rad = 1.5 * u.degree

In [ ]:
#perform the vizier query
pos = coords.SkyCoord(ra=search_ra, dec=search_dec, frame='icrs')
result = Vizier.query_region(pos, radius=search_rad, catalog='SDSS')

ra = result[0]['_RAJ2000']
dec = result[0]['_DEJ2000']

In [ ]:
#pick reasonable SDSS limiting magnitudes and selection cuts to calculate the zeropoint. Also, only select
#stars to do the comparison (SDSS class ==6)
sdss_faint_cut = 20
sdss_bright_cut = 16
filt = 'rmag'

good_sdss_sources = (result[0]['cl'] == 6) & \
    (result[0][filt] < sdss_faint_cut) & (result[0][filt] > sdss_bright_cut)

In [ ]:
#print out the number of good comparisons for testing purposes
np.sum(good_comps)

In [ ]:
#write out a region file containing the 
write_reg(ra[good_comps],dec[good_comps],filename='sdss_bright_r.reg',width=5,size='5"')

In [ ]:
data = np.loadtxt('bg1_r_uncorrected.csv',delimiter=',')

In [ ]:
good_data = (data[:,2] < 23) & (data[:,2] > 14) & (data[:,4] > 2.8) & (data[:,4] < 3.2) & (data[:,6] < 1)

In [ ]:
cut_data = data[good_data,:]
cut_data[:,2] = cut_data[:,2] - 0.33645 - 0.037

In [ ]:
ra_meas = cut_data[:,0]
dec_meas = cut_data[:,1]

In [ ]:
measured_coords = coords.SkyCoord(ra=ra_meas*u.degree,\
    dec=dec_meas*u.degree)

In [ ]:
sdss_coords = coords.SkyCoord(ra=ra[good_comps],dec=dec[good_comps])

In [154]:
full_old,full_new,d2d,d3d = sdss_coords.search_around_sky(measured_coords,0.1*u.arcsec)

In [ ]:
diffs = cut_data[full_old,2] - (result[0][filt][good_comps][full_new])
                      # - 0.085*(result[0]['rmag'][good_comps][full_new] - result[0]['imag'][good_comps][full_new]))

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.scatter(cut_data[full_old,2],diffs,s=2)


### Calculate the final shift. Actual zeropoint is whatever was used to make initial catalog plus the below value.

In [157]:
zpt_shift = np.median(diffs)
print(zpt_shift)

-0.000500175476072
